In [1]:
import os
import cv2
import numpy as np
from PIL import Image, ImageDraw

In [2]:
def draw_bounding_boxes(image, mask):
    """
    Vẽ bounding boxes quanh các object màu đen trong ảnh mask lên ảnh gốc.
    """
    # Chuyển mask sang numpy array
    mask = np.array(mask)

    # Đảo ngược màu (0 là đen -> 255 là trắng, để detect vùng đen)
    mask_inverted = cv2.bitwise_not(mask)

    # Tìm các contours (khu vực liên tục) trong mask
    contours, _ = cv2.findContours(mask_inverted, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Chuyển ảnh gốc sang PIL để vẽ
    image = Image.fromarray(np.array(image))
    draw = ImageDraw.Draw(image)

    # Duyệt qua các contours và vẽ bounding box
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        draw.rectangle([x, y, x + w, y + h], outline="red", width=3)

    return image

TẠO FILE JSON TỪ ẢNH MASK & 
VẼ BOUNDING BOX TỪ FILE JSON

In [ ]:
import os
import json
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import random

def generate_random_color():
    return tuple(random.randint(50, 255) for _ in range(3))

def find_bounding_boxes(mask):
    mask_array = np.array(mask)
    height, width = mask_array.shape
    visited = np.zeros((height, width), dtype=bool)
    bounding_boxes = []

    for y in range(height):
        for x in range(width):
            if mask_array[y, x] > 0 and not visited[y, x]:
                x_min, y_min, x_max, y_max = x, y, x, y
                stack = [(x, y)]
                while stack:
                    px, py = stack.pop()
                    if 0 <= px < width and 0 <= py < height and mask_array[py, px] > 0 and not visited[py, px]:
                        visited[py, px] = True
                        x_min, y_min = min(x_min, px), min(y_min, py)
                        x_max, y_max = max(x_max, px), max(y_max, py)
                        stack.extend([(px+1, py), (px-1, py), (px, py+1), (px, py-1)])
                bounding_boxes.append((x_min, y_min, x_max, y_max))
    return bounding_boxes

def draw_bounding_boxes(image, mask, image_name, json_path):
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    bounding_boxes = find_bounding_boxes(mask)
    objects_info = []

    for idx, (x_min, y_min, x_max, y_max) in enumerate(bounding_boxes, start=1):
        color = generate_random_color()
        draw.rectangle([x_min, y_min, x_max, y_max], outline=color, width=3)
        object_id = f"{image_name}.{idx}"
        text_position = (x_min + 5, y_min + 5)
        draw.text(text_position, object_id, fill=color, font=font)
        objects_info.append({
            "id": object_id,
            "bbox": [x_min, y_min, x_max - x_min, y_max - y_min],
            "color": color
        })
    with open(json_path, "w", encoding="utf-8") as json_file:
        json.dump(objects_info, json_file, ensure_ascii=False, indent=4)
    return image

def process_folder(image_folder, mask_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    image_extensions = {".jpeg", ".jpg", ".png"}
    mask_extensions = {".png"}  

    image_files = {os.path.splitext(f)[0]: f for f in os.listdir(image_folder) if os.path.splitext(f)[1].lower() in image_extensions}
    mask_files = {os.path.splitext(f)[0]: f for f in os.listdir(mask_folder) if os.path.splitext(f)[1].lower() in mask_extensions}

    for image_name, image_file in image_files.items():
        image_path = os.path.join(image_folder, image_file)
        mask_path = os.path.join(mask_folder, mask_files[image_name])
        output_image_path = os.path.join(output_folder, image_file)
        output_json_path = os.path.join(output_folder, f"{image_name}.json")
        
        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")
        
        output_image = draw_bounding_boxes(image, mask, image_name, output_json_path)
        output_image.save(output_image_path)
        print(f"✅ Đã xử lý: {image_file}")

image_folder = "/media/data3/users/huytq/FastAPIDemo/datasets/images"
mask_folder = "/media/data3/users/huytq/FastAPIDemo/datasets/masks"
output_folder = "/media/data3/users/huytq/FastAPIDemo/datasets/output"

process_folder(image_folder, mask_folder, output_folder)
